In [1]:
import pandas as pd
import json
from collections import defaultdict
import re
from utilities import Tiempo
import dataExtraction

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados

In [2]:
def extraerTiempos(rawData, minTiempoInactivo):
    #HAY QUE COMPROBAR QUE minTiempoInactivo es positivo
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erAccessed = re.compile(r'\baccessed$\b')
    erInteracted = re.compile(r'\binteracted$\b')

    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    erSeriousGame = re.compile(r'\bserious-game$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erScreen = re.compile(r'\bscreen$\b')

    erExitGamePanel = re.compile(r'\bexit_game_panel$\b')
    erExitGamePanelCloseButton = re.compile(r'\bexit_game_panel_close_button$\b')
    
    data = defaultdict(defaultdict)
    infoNivel = defaultdict(defaultdict)
    ultimaTrazaRegistrada = defaultdict()
    tiemposInactivo = defaultdict(list)
    inicioYFinJuego = defaultdict(list)
    anomalias = []
    
    for evento in rawData:         
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
        
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if erInitialized.search(verb): #Si la acción es inicio o reinicio
                if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                    if levelCode in data[name]:
                        data[name][levelCode].append({"ini" : timestamp, "fin" : "", "stars" : ""})
                    else:
                        data[name][levelCode] = [{"ini" : timestamp, "fin" : "", "stars" : ""}]
                #else: #-> sería cuando un nivel es reiniciado, por ahora no vamos a tenerlo en cuenta
            elif erCompleted.search(verb):
                if evento["result"]["score"]["raw"] > 0 :
                    dataExtraction.anadirAlDiccionario(infoNivel, evento, name, levelCode)
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = timestamp
                        data[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
                else:
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = None #Porque el jugador sale al menú o cierra el juego
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
        
        #Algunos jugadores cierran el juego el juego con Alt+F4 o dando a la x de la ventana, esto no queda registrado
        #Para poder tener esto en cuenta habría que recorrer los datos 2 veces ya que no podemos quedarnos con el último 
        #valor antes del initialized porque los timestamps no vienen bien ordenados
        
        elif erSeriousGame.search(obj) and erInitialized.search(verb): #Nos indica que el jugador ha abierto el juego
            inicioYFinJuego[name].append({"inicio" : timestamp, "fin" : None, "exit-type": "inesperado"})
            
        elif erAccessed.search(verb) and erScreen.search(obj) and erExitGamePanel.search(objectId): #El jugador ha abierto el panel para cerrar el juego
            inicioYFinJuego[name][-1]["fin"] = timestamp
            inicioYFinJuego[name][-1]["exit-type"] = "normal"
      
    for evento in rawData:
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
                
        for i in range(len(inicioYFinJuego[name])):
            if inicioYFinJuego[name][i]["fin"] == None or inicioYFinJuego[name][i]["exit-type"] == "inesperado": #No hemos conseguido obtener el final porque se ha cerrado el juego
                if (i + 1) < len(inicioYFinJuego[name]): #No estamos en la ultima partida del jugador
                    if Tiempo(timestamp, inicioYFinJuego[name][i+1]["inicio"]) > Tiempo("1s"): #No es una trazas que aparece antes de iniciar el juego
                        inicioYFinJuego[name][i]["fin"] = timestamp
                else:
                    inicioYFinJuego[name][i]["fin"] = timestamp
                    
        if name in ultimaTrazaRegistrada:
                t = Tiempo(ultimaTrazaRegistrada[name], timestamp)
                if t > minTiempoInactivo:
                    tiemposInactivo[name].append(t)
                   
                    for ini in inicioYFinJuego[name]:
                        if(Tiempo(ini["inicio"], timestamp) < Tiempo("1s")):
                            del tiemposInactivo[name][-1] #Eliminamos los tiempos inactivos que sean de distintos inicios del juego
                            break
                        
        ultimaTrazaRegistrada[name] = timestamp
                    
                    
    return {"tiemposNivel" : data, "tiempoTotal" : inicioYFinJuego, "anomalias" : anomalias, "tiemposInactivo" : tiemposInactivo, "infoNivel" : infoNivel}              

In [3]:
def tiempoTotalJuego(inicioYFinJuego):
    tiempoTotal = defaultdict()
    for player in inicioYFinJuego:
        for time in inicioYFinJuego[player]:
            if time["fin"] != None:
                if player in tiempoTotal:
                    tiempoTotal[player] += Tiempo(time["inicio"], time["fin"])
                else:
                    tiempoTotal[player] = Tiempo(time["inicio"], time["fin"])
    return tiempoTotal

In [4]:
def tiempoPorNiveles(data, anomalias):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    try:
                        timeDifference = Tiempo(times["ini"], times["fin"])
                        if level in tiemposJugados[player]:
                            tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                        else:
                            tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
                    except ValueError: #Algunos timestamps de finalizacion estan vacios, ¿Tiene que ser asi?
                        anomalias.append({"jugador" : player,
                                  "nivel" : level,
                                  "Timestamp-Inicio" : times["ini"],
                                  "Timestamp-Fin" : times["fin"],
                                  "Descripcion:" : "No se ha podido parsear uno de los 2 timestamps"})
    return tiemposJugados

In [5]:
def buscarTarjetas(rawData):
    data = []
    erIdLevel = re.compile(r'/')
    for evento in rawData:
        idJugador = evento["actor"]["name"]
        levelCode = erIdLevel.split(evento["object"]["id"])[-1]
        if idJugador == "bdwyj" and "result" in evento and "extensions" in evento["result"] and "level" in evento["result"]["extensions"]:
            if evento["result"]["extensions"]["level"] == "variables_9":
                data.append(evento)
        elif idJugador == "bdwyj" and levelCode == "variables_9":
            data.append(evento)
    return data

In [6]:
def getMediaTiemposYEstrellas(data):
    tiemposYEstrellasMedios = defaultdict()
    sumasTotales = defaultdict()
    for user in data:
        for level in data[user]:
            for time in data[user][level]:
                if level in tiemposYEstrellasMedios:
                    tiemposYEstrellasMedios[level]["tiempo"] += time["time"]
                    tiemposYEstrellasMedios[level]["stars"] += time["stars"]
                    sumasTotales[level] += 1
                else:
                    tiemposYEstrellasMedios[level] = {"tiempo" : time["time"], "stars" : time["stars"]}
                    sumasTotales[level] = 1
    for level in tiemposYEstrellasMedios:
        tiemposYEstrellasMedios[level]["tiempo"] = tiemposYEstrellasMedios[level]["tiempo"]/sumasTotales[level];
        tiemposYEstrellasMedios[level]["stars"] = round(tiemposYEstrellasMedios[level]["stars"]/sumasTotales[level], 2);
    return tiemposYEstrellasMedios

In [7]:
def getDesempenioJugadores(tiempos, tiemposMedios, porcDebajo, porcEncima, primerIntento = True):
    #tiempos -> diccionario con los tiempos de cada jugador en cada nivel
    #tiemporMedios -> media de los tiempos por nivel
    #porcDebajo -> porcentaje que se usara para filtrar a los jugadores con un tiempo igual o inferior a este
    #porcEncima -> porcentaje que se usara para filtrar a los jugadores con un tiempo igual o superior a este
    #primerIntento -> true cuando solo se quiere tener en cuenta el primer intento del jugador, false cuando no
    jugadoresPorDebajo = defaultdict(defaultdict) #Jugadores con buen desempeño, menor tiempo que media
    jugadoresPorEncima = defaultdict(defaultdict) #Jugaodre con mal desempeño, mayor tiempo que media

    for player in tiempos:
        for level in tiempos[player]:
            for time in tiempos[player][level]:
                if time["time"] <= (tiemposMedios[level]["tiempo"]*(1-porcDebajo)):
                    if level in jugadoresPorDebajo[player]:
                        jugadoresPorDebajo[player][level].append(time["time"])
                    else:
                        jugadoresPorDebajo[player][level] = [time["time"]]
                        
                elif time["time"] >= (tiemposMedios[level]["tiempo"]*(1+porcEncima)):
                    if level in jugadoresPorEncima[player]:
                        jugadoresPorEncima[player][level].append(time["time"])
                    else:
                        jugadoresPorEncima[player][level] = [time["time"]]
                        
                if primerIntento:
                    break
                    
    return jugadoresPorDebajo, jugadoresPorEncima

In [8]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [9]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
minTiempoInactivo = Tiempo("1m/30s")
result = extraerTiempos(rawData, minTiempoInactivo)
JSONFile.close()
#pd.DataFrame(result["data"])
#pd.DataFrame.from_dict(result["tiempoTotal"], orient='index').transpose()
#pd.DataFrame.from_dict(result["tiemposInactivo"], orient='index').transpose()
#tiempoTJuego = tiempoTotalJuego(result["tiempoTotal"])
#pd.DataFrame(tiempoTJuego, index = ["Tiempo"])
pd.DataFrame(result["infoNivel"])

,gqoj,hgfx,uala,qeqyj,fkmtp,eprdu,bdwyj,pbgky,hjpbw,ftpnb,loemd,gcpze,gleqe,srjon,khadt,sxipu,ikcar,uzzpo,vsxld,zxvkj,ejeuw,kmqou,xnpqf,ipdcp,epqkm,zwzfv,rdiaf,auvqa,krahz,grkij,zcibf,ubmtx,pygoc,onyep,cmbfi,asopk,yvpmf,oahey,edxmq,bhyhw,accul,tekwp,swvxx,dunul,kkoif,yfohf,dwqwi,tjfzy,nmveq,sjpkc,lwryl,fivin,ewqbs,tinco,psido,zbgwk,fyyya,gdljy,mymho,wjaxi,yxewz,gguuc,mjiso,hpdxj,dvmlu,camhi,kkliq,hqmhq,tgwmn,rziia,ucdvy,dmwvy,ztyed,yxjji,qfsdj,stiry,bjvnt,ngavz,vddlg,atlzs,yazod,tqgyp,idmzi,huyyb,zdnso,lhrbj,tavin,vvqlm,aotlm,mgaak,wqmgq,knewj,tvxow,ahxym,rqlug,jozyx,xtdrq,jhbuv,ptmkr,pzfdd,buvgs,sbexy,thaho,pbwkf,khfha,mewpz,fmtrq,icllt,nhbnj,qwnpf,itkas,uniee,esrit,rqaby,yyvas,kzles,gzgfk,hcbyl,rkmrw,nuasx,nplrf,ixrux,pqwaf,cojqb,qgcgc,lohyo,rfofs,qbsjv,rsjxu,sybwc,eayrh,orjyn,bajbm,fbldj,tcvyz,blvkx,tflzv,raqdt,pleui,xbmmw,prpej,vkfnt,fmupf,olugf,epjcx
tutorials_1,"[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}, {'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}, {'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}, {'pasosMinimos': True, 'pistasNoUsadas': True, 'primeraEjecucion': True, 'estrellas': 3, '3_estrellas': True}]","[{'pasosMinimos': True, 'pistasNoUsadas': True, 'p

In [10]:
tiemposJ = tiempoPorNiveles(result["tiemposNivel"], result["anomalias"])
pd.DataFrame(tiemposJ)

,gqoj,hgfx,uala,qeqyj,fkmtp,bdwyj,eprdu,hjpbw,pbgky,ftpnb,gleqe,ikcar,loemd,gcpze,khadt,zxvkj,srjon,uzzpo,vsxld,sxipu,kmqou,ejeuw,xnpqf,ipdcp,epqkm,zwzfv,krahz,grkij,zcibf,auvqa,rdiaf,yvpmf,pygoc,ubmtx,edxmq,onyep,swvxx,cmbfi,oahey,asopk,kkoif,bhyhw,accul,tekwp,dunul,dwqwi,yfohf,nmveq,tjfzy,sjpkc,ewqbs,lwryl,fivin,psido,tinco,mymho,zbgwk,fyyya,gdljy,wjaxi,ucdvy,dvmlu,mjiso,yxewz,gguuc,kkliq,tgwmn,dmwvy,hpdxj,camhi,hqmhq,rziia,ztyed,yxjji,stiry,vddlg,qfsdj,bjvnt,yazod,ngavz,atlzs,tqgyp,idmzi,aotlm,vvqlm,zdnso,lhrbj,tavin,ahxym,huyyb,mgaak,wqmgq,knewj,rqlug,tvxow,jhbuv,jozyx,xtdrq,ptmkr,pbwkf,thaho,pzfdd,qwnpf,icllt,mewpz,buvgs,fmtrq,nhbnj,sbexy,rqaby,khfha,itkas,gzgfk,yyvas,uniee,esrit,kzles,rkmrw,hcbyl,nuasx,nplrf,ixrux,qgcgc,rfofs,pqwaf,lohyo,cojqb,tflzv,qbsjv,rsjxu,sybwc,blvkx,orjyn,fbldj,raqdt,eayrh,tcvyz,prpej,bajbm,fmupf,xbmmw,pleui,vkfnt,olugf,epjcx
tutorials_1,"[{'time': 19s, 'stars': 3}, {'time': 48s, 'stars': 3}, {'time': 32s, 'stars': 3}]","[{'time': 11s, 'stars': 3}]","[{'time': 40s, 'stars': 3}]","[{'time': 25s, 'stars': 3}]","[{'time': 25s, 'stars': 3}]","[{'time': 57s, 'stars': 3}]","[{'time': 40s, 'stars': 3}]","[{'time': 52s, 'stars': 3}]","[{'time': 26s, 'stars': 3}]","[{'time': 25s, 'stars': 3}]","[{'time': 41s, 'stars': 3}]","[{'time': 1m/6s, 'stars': 3}]","[{'time': 17s, 'stars': 3}]","[{'time': 19s, 'stars': 3}]","[{'time': 42s, 'stars': 3}]","[{'time': 1m, 'stars': 3}]","[{'time': 35s, 'stars': 3}]","[{'time': 40s, 'stars': 3}]","[{'time': 42s, 'stars': 3}]","[{'time': 28s, 'stars': 3}]","[{'time': 1m/22s, 'stars': 3}]","[{'time': 33s, 'stars': 3}]","[{'time': 39s, 'stars': 3}]","[{'time': 29s, 'stars': 3}, {'time': 23s, 'stars': 3}]","[{'time': 26s, 'stars': 3}]","[{'time': 33s, 'stars': 3}]","[{'time': 1m/3s, 'stars': 3}]","[{'time': 1m/11s, 'stars': 3}]","[{'time': 1m/14s, 'stars': 3}]","[{'time': 29s, 'stars': 3}]","[{'time': 20s, 'stars': 3}]","[{'time': 1m/27s, 'stars': 3}]","[{'time': 31s, 'stars': 3}]","[{'time': 22s, 'stars': 3}]","[{'time': 1m/8s, 'stars': 3}]","[{'time': 27s, 'stars': 3}]","[{'time': 55s, 'stars': 3}]","[{'time': 17s, 'stars': 3}]","[{'time': 29s, 'stars': 3}]","[{'time': 25s, 'stars': 3}]","[{'time': 1m/5s, 'stars': 3}]","[{'time': 37s, 'stars': 3}]","[{'time': 21s, 'stars': 3}]","[{'time': 25s, 'stars': 3}]","[{'time': 26s, 'stars': 3}]","[{'time': 1m/12s, 'stars': 3}]","[{'time': 34s, 'stars': 3}]","[{'time': 1m/11s, 'stars': 3}]","[{'time': 24s, 'stars': 3}]","[{'time': 1m/3s, 'stars': 3}]","[{'time': 1m, 'stars': 3}]","[{'time': 32s, 'stars': 3}]","[{'time': 36s, 'stars': 3}]","[{'time': 56s, 'stars': 3}]","[{'time': 29s, 'stars': 3}]","[{'time': 1m/16s, 'stars': 3}]","[{'time': 39s, 'stars': 3}]","[{'time': 37s, 'stars': 3}]","[{'time': 32s, 'stars': 3}]","[{'time': 31s, 'stars': 3}]","[{'time': 2m/34s, 'stars': 3}]","[{'time': 1m/3s, 'stars': 3}]","[{'time': 47s, 'stars': 3}]","[{'time': 27s, 'stars': 3}]","[{'time': 36s, 'stars': 3}]","[{'time': 1m/4s, 'stars': 3}]","[{'time': 1m/20s, 'stars': 3}]","[{'time': 2m, 'stars': 3}]","[{'time': 24s, 'stars': 3}]","[{'time': 35s, 'stars': 3}]","[{'time': 37s, 'stars': 3}]","[{'time': 25s, 'stars': 3}]","[{'time': 1m/8s, 'stars': 3}]","[{'time': 1m/17s, 'stars': 3}]","[{'time': 1m/11s, 'stars': 3}]","[{'time': 1m/32s, 'stars': 3}]","[{'time': 21s, 'stars': 3}]","[{'time': 41s, 'stars': 3}]","[{'time': 32s, 'stars': 3}]","[{'time': 14s, 'stars': 3}]","[{'time': 21s, 'stars': 3}]","[{'time': 14s, 'stars': 3}]","[{'time': 35s, 'stars': 3}]","[{'time': 1m/13s, 'stars': 3}]","[{'time': 49s, 'stars': 3}]","[{'time': 32s, 'stars': 3}]","[{'time': 33s, 'stars': 3}]","[{'time': 32s, 'stars': 3}]","[{'time': 2m/34s, 'stars': 3}]","[{'time': 18s, 'stars': 3}]","[{'time': 51s, 'stars': 3}]","[{'time': 1m/29s, 'stars': 3}]","[{'time': 1m/1s, 'stars': 3}]","[{'time': 2m/1s, 'stars': 3}]","[{'time': 21s, 'stars': 3}]","[{'time': 1m/28s, 'stars': 3}]","[{'time': 34s, 'stars': 3}]","[{'time': 28s, 'stars': 3}]","[{'time': 46s, 'stars': 3}]","[{'

In [11]:
tiemposMedios = getMediaTiemposYEstrellas(tiemposJ)
pd.DataFrame(tiemposMedios)

,tutorials_1,tutorials_2,tutorials_3,tutorials_4,tutorials_5,tutorials_6,tutorials_7,variables_1,variables_2,variables_3,variables_4,variables_5,variables_6,variables_7,variables_8,variables_9,types_1,types_2,types_3,types_4,types_5,types_6,types_7,types_8,types_9,basic_operators_1,basic_operators_2,basic_operators_3,basic_operators_4,basic_operators_5,basic_operators_6,basic_operators_7,loops_1,loops_2,loops_3
tiempo,57s,52s,36s,3m/1s,32s,2m/13s,2m/13s,1m,4m/21s,2m/52s,3m/54s,5m/39s,4m/22s,3m/44s,3m/1s,2m/53s,1m/37s,1m/35s,5m/1s,5m/45s,6m/27s,3m/38s,4m/20s,3m/49s,3m/27s,46s,6m/43s,5m/30s,8m/46s,2m/52s,2m/4s,2m/4s,2m/40s,1m/19s,1m/46s
stars,3.0,2.91,3.0,3.0,2.99,2.89,2.87,3.0,2.9,2.82,2.82,2.81,2.24,2.9,2.91,2.82,2.9,2.94,2.83,2.86,2.86,2.86,2.95,2.98,2.95,3.0,2.9,2.75,2.83,3.0,2.86,3.0,3.0,3.0,2.0


In [12]:
porcentajePorDebajo = 0.2
porcentajePorEncima = 0.5
jPorDebajo, jPorEncima = getDesempenioJugadores(tiemposJ, tiemposMedios, porcentajePorDebajo, porcentajePorEncima)
pd.DataFrame(jPorEncima)

,qeqyj,fkmtp,bdwyj,eprdu,hjpbw,ftpnb,gleqe,ikcar,khadt,zxvkj,srjon,uzzpo,kmqou,ejeuw,xnpqf,zwzfv,krahz,grkij,auvqa,rdiaf,yvpmf,pygoc,ubmtx,edxmq,onyep,swvxx,cmbfi,oahey,asopk,kkoif,bhyhw,accul,tekwp,dunul,dwqwi,yfohf,nmveq,tjfzy,sjpkc,ewqbs,lwryl,fivin,psido,tinco,mymho,zbgwk,fyyya,gdljy,wjaxi,ucdvy,dvmlu,mjiso,yxewz,gguuc,kkliq,tgwmn,dmwvy,hpdxj,camhi,hqmhq,rziia,ztyed,yxjji,vddlg,bjvnt,yazod,ngavz,atlzs,idmzi,aotlm,vvqlm,zdnso,lhrbj,tavin,ahxym,huyyb,mgaak,wqmgq,knewj,rqlug,tvxow,jhbuv,jozyx,xtdrq,ptmkr,pbwkf,thaho,pzfdd,qwnpf,icllt,mewpz,buvgs,fmtrq,nhbnj,sbexy,rqaby,khfha,itkas,gzgfk,yyvas,uniee,esrit,kzles,rkmrw,hcbyl,nuasx,nplrf,ixrux,qgcgc,rfofs,pqwaf,lohyo,cojqb,tflzv,qbsjv,rsjxu,sybwc,blvkx,orjyn,fbldj,raqdt,eayrh,tcvyz,prpej,bajbm,fmupf,xbmmw,pleui,vkfnt,olugf,epjcx
variables_4,[10m/19s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[11m/6s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[14m/11s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[9m/46s],[6m/21s],NaN,NaN,NaN,NaN,NaN,[6m/40s],NaN,NaN,[6m/17s],NaN,[10m/19s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[7m/13s],NaN,NaN,NaN,[6m/17s],[8m/40s],NaN,[5m/53s],NaN,[6m/36s],NaN,[6m/54s],NaN,[6m/55s],NaN,NaN,NaN,NaN,NaN,NaN,[8m/50s],NaN,NaN,NaN,NaN,NaN,[6m/19s],NaN,NaN,NaN,NaN,NaN,NaN,[8m/16s],[5m/56s],NaN,NaN,NaN,[6m/18s],NaN,NaN,NaN,NaN,[16m/3s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[7m/25s],[7m/46s],[11m/7s],NaN,NaN,[8m/34s],[9m/4s],[5m/59s],[15m/45s],NaN
types_5,[13m/19s],NaN,NaN,NaN,NaN,NaN,NaN,[12m/16s],NaN,NaN,NaN,NaN,[11m/33s],[10m/53s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10m/55s],NaN,NaN,NaN,NaN,NaN,NaN,[11m/46s],NaN,NaN,NaN,NaN,NaN,[9m/46s],NaN,NaN,NaN,NaN,NaN,NaN,[13m/54s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[18m/24s],[11m/39s],[13m/26s],NaN,NaN,NaN,NaN,NaN,[12m/13s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
types_7,[6m/41s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[8m/36s],[6m/44s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[6m/36s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[8m/26s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[7m/45s],NaN,NaN,NaN,NaN,[6m/30s],NaN,NaN,NaN,NaN
types_9,NaN,[5m/10s],NaN,NaN,NaN,NaN,[6m/36s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[9m/12s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tutorials_5,NaN,NaN,[2m/55s],NaN,[50s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1m/29s],NaN,[55s],NaN,NaN,[55s],[55s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1m/13s],[1m],NaN,NaN,NaN,[1m/7s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1m/3s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1m/13s],NaN,NaN,NaN,NaN,NaN,[1m/10s],[1m/1s],[48s],NaN,NaN,NaN,NaN,[1m/9s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1m/57s],NaN,NaN,NaN,NaN,[1m/20s],NaN,NaN,NaN,NaN,NaN,NaN,[54s],NaN,NaN,[59s],NaN,NaN,NaN,NaN,NaN,NaN
tutorials_7,NaN,NaN,[3m/35s],NaN,[3m/56s],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[3m/24s],NaN,[3m/21s],[3m/56s],NaN,NaN,NaN,NaN,NaN,Na

In [13]:
Tiempo("2022-05-18T07:13:48.865Z", "2022-05-18T06:13:48.865Z").positivo

True